In [1]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.parser import parse

In [2]:
# working directory
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/California/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
# Input files
fileInput1 = "EWRIMS MASTER FLAT FILE DATA DICTIONARY DRAFT 1-17-20.xlsx" 
# water sources look up
inp_wtrsrs="watersources.csv"
# sites look up
inp_sitdim = 'sites.csv'

#output: water allocation
out_alloc = "waterallocations.csv"    #output

In [4]:
######## WaDE columns

#the followwing fields have difference between the table here (edited by DPL) and that on the schema website
#http://schema.westernstateswater.org/tables/Input_AllocationAmounts_fact.html
"""
BeneficialUseCategory, PrimaryUseCategory, AllocationTimeframeStart, AllocationTimeframeEnd, " "
BeneficialUseCategoryCV, PrimaryUseCategoryCV, TimeframeStartDate,	TimeframeEndDate,	Geometry	
"""
# UUIDs: Add UUIDs for all dim tables
# OrganizationUUID, SiteUUID, VariableSpecificUUID, WaterSourceUUID, MethodUUID
columns = ["OrganizationUUID", "SiteUUID", "VariableSpecificUUID", "WaterSourceUUID", "MethodUUID", "PrimaryUseCategory",
           "BeneficialUseCategory", "AllocationNativeID", "AllocationTypeCV", "AllocationOwner",
           "AllocationApplicationDate", "AllocationPriorityDate", "AllocationLegalStatusCV", "AllocationCropDutyAmount",
           "AllocationExpirationDate",
           "AllocationChangeApplicationIndicator", "LegacyAllocationIDs", "AllocationBasisCV", "AllocationTimeframeStart",
           "AllocationTimeframeEnd", "AllocationAmount", "AllocationMaximum", "PopulationServed", "PowerType", "GeneratedPowerCapacityMW",
           "IrrigatedAcreage", "AllocationCommunityWaterSupplySystem", "AllocationSDWISIdentifierCV",
           "AllocationAssociatedWithdrawalSiteIDs", "AllocationAssociatedConsumptiveUseSiteIDs", "WaterAllocationNativeURL",
           "CustomerTypeCV", "IrrigationMethodCV", "CropTypeCV", "CommunityWaterSupplySystem", "DataPublicationDate",
           "DataPublicationDOI"]

dtypesx = [''] #here we could theoretically specify data types for each column name, but we didn't need to do that

In [5]:
### target dataFrame

# TODO: assumes dtypes inferred from CO file
outdf100=pd.DataFrame(columns=columns)

In [6]:
print("Reading inputs...")

# 
# sheet to read = "ewrims_flat_file"
df100 = pd.read_excel(fileInput1, header=0, sheet_name="ewrims_flat_file", skiprows=0, encoding = "ISO-8859-1") #, or alternatively encoding = "utf-8"
#print (len(df100.index))
#df100


# sites look up
df500 = pd.read_csv(inp_sitdim, encoding = "ISO-8859-1")

# water sources look up
df400 = pd.read_csv(inp_wtrsrs, encoding = "ISO-8859-1")

Reading inputs...


C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# use only unique water rights that may have multiple sites/pds
print("Dropping duplicates...")

print (len(df100.index))

df100.drop_duplicates(subset = ['APPLICATION_NUMBER'], inplace=True)   #
df100 = df100.reset_index(drop=True)

print (len(df100.index))

df100 = df100.replace(np.nan, '')

df100

Dropping duplicates...
57736
57715


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,PETITION_STATUS_TYPE,DATE_RECEIVED,DATE_COMPLETED,PET_LAST_UPDATE_DATE,NUM_OF_PETITIONS,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE
0,0,,,,,Not Determined,,,,,...,,,,,,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,,,,,,0
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,,,,,,0
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,,,,,,0
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,Pending,1578441600000000000,,1578564981000000000,34,,,,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,,,,,,0
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,,,,,,0
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,,,,,,0
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,,,,,,0


In [8]:
print("Adding SiteUUID...")

def assignSiteID(colrowValue, df500):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        sitl = df500.loc[df500['SiteNativeID'] == colrowValue, 'SiteUUID']
        #print(sitl)
        #print(sitl.empty)
        if not(sitl.empty):            # check if the series is empty
            outList = ', '.join(str(inx) for inx in sitl) #sil.iloc[0]
        else:
            outList = ''
    return outList

df100 = df100.assign(SiteUUID='')  #add new column and make is nan

#Permit Number
df100['SiteUUID'] = df100.apply(lambda row: assignSiteID(row['POD_ID'], df500), axis=1)
#pod_location_id
df100

Adding SiteUUID...


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,DATE_RECEIVED,DATE_COMPLETED,PET_LAST_UPDATE_DATE,NUM_OF_PETITIONS,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID
0,0,,,,,Not Determined,,,,,...,,,,,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,,,,,0,CA_60498
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,,,,,0,CA_34881
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,,,,,0,CA_28036
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,1578441600000000000,,1578564981000000000,34,,,,,0,CA_23233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,,,,,0,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,,,,,0,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,,,,,0,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,,,,,0,


In [9]:
print("Water sources...")

def assignWaterSourceID(colrowValue11, df400):
    colrowValue1 = str(colrowValue11).strip()
    if ((colrowValue1 == '') | (pd.isnull(colrowValue1))):
        outList = ''
    else:
        ml = df400.loc[df400['WaterSourceName'] == colrowValue1, 'WaterSourceUUID']
        #print(ml)
        #print(ml.empty)
        if not(ml.empty):            # check if the series is empty
            outList = ml.iloc[0]   # watersourceSer.append(ml.iloc[0])
        else:
            outList = ''
    return outList

df100 = df100.assign(WaterSourceUUID='')

df100['WaterSourceUUID'] = df100.apply(lambda row: 
                        assignWaterSourceID(row['SOURCE_NAME'], df400), axis=1)

df100

Water sources...


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,DATE_COMPLETED,PET_LAST_UPDATE_DATE,NUM_OF_PETITIONS,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID,WaterSourceUUID
0,0,,,,,Not Determined,,,,,...,,,,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98,,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,,,,0,CA_60498,CA_2
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,,,,0,CA_34881,CA_3
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,,,,0,CA_28036,CA_4
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,,1578564981000000000,34,,,,,0,CA_23233,CA_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,,,,0,,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,,,,0,,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,,,,0,,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,,,,0,,


In [10]:
print("Allocation priority date...")

# input format 1924-12-06 00:00:00 format
def formatDateString(inString1):
    #print(inString1)
    inString = str(inString1).strip()
    #print(inString)
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:
            valD = pd.to_datetime(inString) # Also valD = parse(inString) #--datetuil.parser.parse
            #valD = datetime.strptime(inString, '%Y-%m-%d 00:00:00')
            #print(valD)
            valnDd = valD.date()
            #print(valnDd)
            valndf = valnDd.strftime('%m/%d/%Y')
            #print('date:', valndf)
    except:
        valndf = ''

    return valndf

df100 = df100.assign(AllocationPriorityDate='')

df100['AllocationPriorityDate'] = df100.apply(lambda row: 
                         formatDateString(row['PRIORITY_DATE']) if str(row['PRIORITY_DATE']) != ''
                         else formatDateString(row['APPLICATION_ACCEPTANCE_DATE']), axis=1)

df100

Allocation priority date...


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,PET_LAST_UPDATE_DATE,NUM_OF_PETITIONS,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID,WaterSourceUUID,AllocationPriorityDate
0,0,,,,,Not Determined,,,,,...,,,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98,,,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,,,0,CA_60498,CA_2,04/18/2013
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,,,0,CA_34881,CA_3,02/15/1915
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,,,0,CA_28036,CA_4,03/03/1915
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,1578564981000000000,34,,,,,0,CA_23233,CA_5,03/27/1915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,,,0,,,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,,,0,,,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,,,0,,,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,,,0,,,


In [11]:
print("Timeframe start and time frame end...")

def formatDateString2(inString1):
    #print(inString1)
    inString = str(inString1).strip()
    #print(inString)
    try:
        if inString == '' or pd.isnull(inString):
            valndf = ''
        else:            
            valD = pd.to_datetime(inString) # Also valD = parse(inString) #--datetuil.parser.parse
            #print(valD)
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d')
            #print(valndf)
    except:
        valndf = ''

    return valndf

Timeframe start and time frame end...


In [12]:
print("Timeframe start...")

df100 = df100.assign(AllocationTimeframeStart='')

df100['AllocationTimeframeStart'] = df100.apply(lambda row: 
                                        formatDateString2(row['DIRECT_DIV_SEASON_START']),
                                        axis=1)
df100

Timeframe start...


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,NUM_OF_PETITIONS,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID,WaterSourceUUID,AllocationPriorityDate,AllocationTimeframeStart
0,0,,,,,Not Determined,,,,,...,,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98,,,,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,,0,CA_60498,CA_2,04/18/2013,01/01
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,,0,CA_34881,CA_3,02/15/1915,01/01
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,,0,CA_28036,CA_4,03/03/1915,03/01
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,34,,,,,0,CA_23233,CA_5,03/27/1915,04/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,,0,,,,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,,0,,,,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,,0,,,,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,,0,,,,


In [13]:
print("Timeframe end...")

df100 = df100.assign(AllocationTimeframeEnd='')

df100['AllocationTimeframeEnd'] = df100.apply(lambda row: 
                                            formatDateString2(row['DIRECT_DIV_SEASON_END']),
                                            axis=1)
df100

Timeframe end...


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,ENF_CASE_NUMBER,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID,WaterSourceUUID,AllocationPriorityDate,AllocationTimeframeStart,AllocationTimeframeEnd
0,0,,,,,Not Determined,,,,,...,ENF03549,1573084800000000000,1577750400000000000,Santa Clara,98,,,,,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,,0,CA_60498,CA_2,04/18/2013,01/01,12/31
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,,0,CA_34881,CA_3,02/15/1915,01/01,12/31
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,,0,CA_28036,CA_4,03/03/1915,03/01,11/01
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,,,,,0,CA_23233,CA_5,03/27/1915,04/01,07/01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,,0,,,,,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,,0,,,,,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,,0,,,,,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,,0,,,,,


In [14]:
print("Allocation amount adjust units")

df100 = df100.assign(AllocationAmount='')

def assignAllocAmount(inString1, inString2):
    #Gallons Per Minute (GPM) to CFS
    convFact1 = 0.00222800926
    #Gallons per day (GPD) to CFS
    convFact2 = 1.5472e-6
    ##########
    rowVal = str(inString1).strip()
    unitVal = str(inString2).strip()
    if rowVal == '':
        retVal = ''
    else:
        if unitVal =='Gallons per Minute':
            retVal = float(rowVal) * convFact1
        elif unitVal =='Gallons per Day':
            retVal = float(rowVal) * convFact2
        else:        #unitVal =='Cubic Feet per Second' OR empty (?)
            retVal = float(rowVal)
    
    return retVal

df100['AllocationAmount'] = df100.apply(lambda row: 
                                        assignAllocAmount(row['MAX_RATE_OF_DIVERSION'], 
                                                          row['MAX_RATE_OF_DIV_UNIT']),
                                        axis=1)

df100

Allocation amount adjust units


,WR_WATER_RIGHT_ID,APPLICATION_NUMBER,CERTIFICATE_ID,PERMIT_ID,LICENSE_ID,WATER_RIGHT_TYPE,WATER_RIGHT_STATUS,APPLICATION_NUMBER_PARTY,PWSS_ID,PRIORITY_DATE,...,ENF_CASE_START_DATE,ENF_CASE_CLOSURE_DATE,ENF_CASE_COUNTY,NUMBER_OF_ENFORCEMENT_CASE,SiteUUID,WaterSourceUUID,AllocationPriorityDate,AllocationTimeframeStart,AllocationTimeframeEnd,AllocationAmount
0,0,,,,,Not Determined,,,,,...,1573084800000000000,1577750400000000000,Santa Clara,98,,,,,,
1,1,T032025,,,,Temporary Permit,Cancelled,T032025,,,...,,,,0,CA_60498,CA_2,04/18/2013,01/01,12/31,
2,2,A000016,41,30,41,Appropriative,Licensed,A000016,,,...,,,,0,CA_34881,CA_3,02/15/1915,01/01,12/31,
3,3,A000018,2871,29,2871,Appropriative,Licensed,A000018,,,...,,,,0,CA_28036,CA_4,03/03/1915,03/01,11/01,
4,4,A000023,1986,273,1986,Appropriative,Licensed,A000023,,,...,,,,0,CA_23233,CA_5,03/27/1915,04/01,07/01,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,73078,UN002198,,,,Not Determined,Active,UN002198,,,...,,,,0,,,,,,
57711,73088,UN002199,,,,Not Determined,Active,UN002199,,,...,,,,0,,,,,,
57712,73089,A033099,,,,Appropriative,Pending,A033099,,,...,,,,0,,,,,,
57713,73108,UN002200,,,,Not Determined,Active,UN002200,,,...,,,,0,,,,,,


In [15]:
print("Copying all columns...")
#
destCols=["SiteUUID", "WaterSourceUUID", 
          "AllocationNativeID",  
          "BeneficialUseCategory", 
          "AllocationOwner", 
          "AllocationTypeCV", 
          #"AllocationApplicationDate", 
          "AllocationPriorityDate",
          "AllocationLegalStatusCV",
          "IrrigatedAcreage",
          "AllocationTimeframeStart", "AllocationTimeframeEnd",
          "AllocationAmount", 
          "AllocationMaximum",           
          #"AllocationCropDutyAmount", "AllocationExpirationDate",           
          #"WaterAllocationNativeURL"
         ]
#
srsCols=["SiteUUID", "WaterSourceUUID", 
          "APPLICATION_NUMBER", 
          "USE_CODE", 
          "PRIMARY_OWNER_NAME",
          "WATER_RIGHT_TYPE", 
          #"AllocationApplicationDate",
          "AllocationPriorityDate", 
          "WATER_RIGHT_STATUS",
          "USE_NET_ACREAGE",
          "AllocationTimeframeStart", "AllocationTimeframeEnd",
          "AllocationAmount",
          "USE_DIRECT_DIV_ANNUAL_AMOUNT",          
          #"IRRIGATION_DEPLETION", "DATE_TERMINATED",          
          #"wris_link"
         ]

outdf100[destCols] = df100[srsCols]

outdf100

Copying all columns...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,NaN,,NaN,,NaN,NaN,,,Not Determined,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,CA_60498,NaN,CA_2,NaN,NaN,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,CA_34881,NaN,CA_3,NaN,NaN,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,CA_28036,NaN,CA_4,NaN,NaN,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,CA_23233,NaN,CA_5,NaN,NaN,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,NaN,,NaN,,NaN,NaN,,UN002198,Not Determined,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57711,NaN,,NaN,,NaN,NaN,,UN002199,Not Determined,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57712,NaN,,NaN,,NaN,NaN,,A033099,Appropriative,OMOCHUMNE-HARTNELL WATER DISTRICT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57713,NaN,,NaN,,NaN,NaN,,UN002200,Not Determined,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# hard coded
print("Hard coded...")
#hard coded
outdf100.OrganizationUUID = "CSWRCB"
outdf100.VariableSpecificUUID = "CSWRCB_Allocation All"
outdf100.MethodUUID = "CSWRCB_Water Rights"
outdf100.AllocationBasisCV = "Unknown"
# check this later
outdf100.PrimaryUseCategory = "Irrigation"
#
#outdf100.AllocationTimeframeStart = "01/01"
#outdf100.AllocationTimeframeEnd = "12/31"

#
outdf100.DataPublicationDate = datetime.now().strftime('%m/%d/%Y') 
#"10/31/2019" # edit this to the code run date

outdf100 = outdf100.replace(np.nan, '')

outdf100

Hard coded...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,,CSWRCB_Allocation All,,CSWRCB_Water Rights,Irrigation,,,Not Determined,,...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57710,CSWRCB,,CSWRCB_Allocation All,,CSWRCB_Water Rights,Irrigation,,UN002198,Not Determined,,...,,,,,,,,,08/06/2020,
57711,CSWRCB,,CSWRCB_Allocation All,,CSWRCB_Water Rights,Irrigation,,UN002199,Not Determined,,...,,,,,,,,,08/06/2020,
57712,CSWRCB,,CSWRCB_Allocation All,,CSWRCB_Water Rights,Irrigation,,A033099,Appropriative,OMOCHUMNE-HARTNELL WATER DISTRICT,...,,,,,,,,,08/06/2020,
57713,CSWRCB,,CSWRCB_Allocation All,,CSWRCB_Water Rights,Irrigation,,UN002200,Not Determined,,...,,,,,,,,,08/06/2020,


In [17]:
print("Droping null allocations...")
# if both Allocation amount and Allocation maximum are empty drop row and save it to a Allocations_missing.csv
outdf100 = outdf100.replace(np.nan, '') #replace blank strings by NaN,
outdf100purge = outdf100.loc[(outdf100["AllocationAmount"] == '') & (outdf100["AllocationMaximum"] == '')]
if len(outdf100purge.index) > 0:
    outdf100purge.to_csv('waterallocations_missing.csv')    #index=False,
    dropIndex = outdf100.loc[(outdf100["AllocationAmount"] == '') & (outdf100["AllocationMaximum"] == '')].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100
#outdf100purge

Droping null allocations...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40159,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
40160,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
40161,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
40162,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [18]:
print("Droping null SiteUUIDs...")
outdf100nullID = outdf100.loc[outdf100["SiteUUID"] == '']
if len(outdf100nullID.index) > 0:
    dropIndex = outdf100.loc[outdf100["SiteUUID"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100

Droping null SiteUUIDs...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39769,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
39770,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
39771,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
39772,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [19]:
print("Droping null Priority date...")
outdf100nullPR = outdf100.loc[outdf100["AllocationPriorityDate"] == '']
if len(outdf100nullPR.index) > 0:
    dropIndex = outdf100.loc[outdf100["AllocationPriorityDate"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)

outdf100
#outdf100nullPR

Droping null Priority date...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38280,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
38281,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
38282,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
38283,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [20]:
print("Droping null WaterSourceUUID ...")
outdf100nullPR = outdf100.loc[outdf100["WaterSourceUUID"] == '']
if len(outdf100nullPR.index) > 0:
    dropIndex = outdf100.loc[outdf100["WaterSourceUUID"] == ''].index
    outdf100 = outdf100.drop(dropIndex)
    outdf100 = outdf100.reset_index(drop=True)
outdf100

Droping null WaterSourceUUID ...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37778,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37779,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37780,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37781,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [21]:
print("Droping duplicates...")
#drop duplicate rows; just make sure
outdf100Duplicated=outdf100.loc[outdf100.duplicated()]
if len(outdf100Duplicated.index) > 0:
    outdf100Duplicated.to_csv("waterallocations_duplicaterows.csv")  # index=False,
    outdf100.drop_duplicates(inplace=True)   #
    outdf100 = outdf100.reset_index(drop=True)

outdf100

Droping duplicates...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37778,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37779,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37780,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37781,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [22]:
print("Checking required is not null...")
# check if any cell of these columns is null
requiredCols = ["OrganizationUUID", "VariableSpecificUUID", "WaterSourceUUID", 
                "MethodUUID", "AllocationPriorityDate"] #SiteUUID
# outdf100_nullMand = outdf100.loc[outdf100.isnull().any(axis=1)] --for all cols
# outdf100_nullMand = outdf100.loc[outdf100[requiredCols].isnull().any(axis=1)]
#(outdf100["SiteUUID"].isnull()) |
outdf100_nullMand = outdf100.loc[(outdf100["OrganizationUUID"] == '') |
                                (outdf100["VariableSpecificUUID"] == '') |
                                (outdf100["WaterSourceUUID"] == '') |
                                (outdf100["MethodUUID"] == '') |
                                (outdf100["AllocationPriorityDate"] == '')]
#outdf100_nullMand = outdf100.loc[[False | (outdf100[varName].isnull()) for varName in requiredCols]]
if(len(outdf100_nullMand.index) > 0):
    outdf100_nullMand.to_csv('waterallocations_mandatoryFieldMissing.csv')  # index=False,
#ToDO: purge these cells if there is any missing? #For now left to be inspected
#outdf100_nullMand

Checking required is not null...


In [23]:
print("Fields longer than 100 chars...")

tarCols = ["OrganizationUUID", "SiteUUID", "VariableSpecificUUID", "WaterSourceUUID", "MethodUUID", "PrimaryUseCategory",
           "BeneficialUseCategory", "AllocationNativeID", "AllocationTypeCV", "AllocationOwner",
           "AllocationApplicationDate", "AllocationPriorityDate", "AllocationLegalStatusCV", "AllocationCropDutyAmount",
           "AllocationExpirationDate",
           "AllocationChangeApplicationIndicator", "LegacyAllocationIDs", "AllocationBasisCV", "AllocationTimeframeStart",
           "AllocationTimeframeEnd", "AllocationAmount", "AllocationMaximum", "PopulationServed", "PowerType", "GeneratedPowerCapacityMW",
           "IrrigatedAcreage", "AllocationCommunityWaterSupplySystem", "AllocationSDWISIdentifierCV",
           "AllocationAssociatedWithdrawalSiteIDs", "AllocationAssociatedConsumptiveUseSiteIDs", "WaterAllocationNativeURL",
           "CustomerTypeCV", "IrrigationMethodCV", "CropTypeCV", "CommunityWaterSupplySystem", "DataPublicationDate",
           "DataPublicationDOI"]

for colVal in tarCols:
    print(colVal)
    outdf100Long = outdf100[outdf100[colVal].apply(lambda x: len(str(x)) > 100)]
    if len(outdf100Long.index) > 0:
        print("There are rows with > 100 chars in "+colVal)

Fields longer than 100 chars...
OrganizationUUID
SiteUUID
VariableSpecificUUID
WaterSourceUUID
MethodUUID
PrimaryUseCategory
BeneficialUseCategory
AllocationNativeID
AllocationTypeCV
AllocationOwner
AllocationApplicationDate
AllocationPriorityDate
AllocationLegalStatusCV
AllocationCropDutyAmount
AllocationExpirationDate
AllocationChangeApplicationIndicator
LegacyAllocationIDs
AllocationBasisCV
AllocationTimeframeStart
AllocationTimeframeEnd
AllocationAmount
AllocationMaximum
PopulationServed
PowerType
GeneratedPowerCapacityMW
IrrigatedAcreage
AllocationCommunityWaterSupplySystem
AllocationSDWISIdentifierCV
AllocationAssociatedWithdrawalSiteIDs
AllocationAssociatedConsumptiveUseSiteIDs
WaterAllocationNativeURL
CustomerTypeCV
IrrigationMethodCV
CropTypeCV
CommunityWaterSupplySystem
DataPublicationDate
DataPublicationDOI


In [24]:
##### Do not run the following with the rest of the code  (it is for inspection)
print("Long site ids...")

print (len(outdf100.index))

outdf100Long = outdf100[outdf100['SiteUUID'].apply(lambda x: len(x) > 250)]
longSiteIDs = False
if len(outdf100Long.index) > 0:
    print("There are rows with too long siteids")
    outdf100Long.to_csv("waterallocations_longsiteid.csv")  # index=False,
    longSiteIDs = True
#outdf100

outdf100Long

Long site ids...
37783


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI


In [25]:
##### Do not run the following with the rest of the code  (it is for inspection)
print("Long site ids remove rows...")

if longSiteIDs:
    print("There are rows with too long siteids")
    dropIndex = outdf100[outdf100['SiteUUID'].apply(lambda x: len(x) > 250)].index
    outdf100 = outdf100.drop(dropIndex)   #
    outdf100 = outdf100.reset_index(drop=True)

outdf100

Long site ids remove rows...


,OrganizationUUID,SiteUUID,VariableSpecificUUID,WaterSourceUUID,MethodUUID,PrimaryUseCategory,BeneficialUseCategory,AllocationNativeID,AllocationTypeCV,AllocationOwner,...,AllocationSDWISIdentifierCV,AllocationAssociatedWithdrawalSiteIDs,AllocationAssociatedConsumptiveUseSiteIDs,WaterAllocationNativeURL,CustomerTypeCV,IrrigationMethodCV,CropTypeCV,CommunityWaterSupplySystem,DataPublicationDate,DataPublicationDOI
0,CSWRCB,CA_60498,CSWRCB_Allocation All,CA_2,CSWRCB_Water Rights,Irrigation,Dust Control,T032025,Temporary Permit,"569 EAST COUNTY BOULEVARD, LLC",...,,,,,,,,,08/06/2020,
1,CSWRCB,CA_34881,CSWRCB_Allocation All,CA_3,CSWRCB_Water Rights,Irrigation,Domestic,A000016,Appropriative,LILAC HILLS ESTATES LP,...,,,,,,,,,08/06/2020,
2,CSWRCB,CA_28036,CSWRCB_Allocation All,CA_4,CSWRCB_Water Rights,Irrigation,Irrigation,A000018,Appropriative,GLENN-COLUSA IRRIGATION DISTRICT,...,,,,,,,,,08/06/2020,
3,CSWRCB,CA_23233,CSWRCB_Allocation All,CA_5,CSWRCB_Water Rights,Irrigation,Domestic,A000023,Appropriative,U.S. BUREAU OF RECLAMATION,...,,,,,,,,,08/06/2020,
4,CSWRCB,CA_405,CSWRCB_Allocation All,CA_6,CSWRCB_Water Rights,Irrigation,Domestic,A000026,Appropriative,"JAMES S PHELPS, TRUSTEE",...,,,,,,,,,08/06/2020,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37778,CSWRCB,CA_77460,CSWRCB_Allocation All,CA_8513,CSWRCB_Water Rights,Irrigation,Power,S028311,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37779,CSWRCB,CA_77461,CSWRCB_Allocation All,CA_8206,CSWRCB_Water Rights,Irrigation,Power,S028312,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37780,CSWRCB,CA_77462,CSWRCB_Allocation All,CA_8611,CSWRCB_Water Rights,Irrigation,Power,S028313,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,
37781,CSWRCB,CA_77465,CSWRCB_Allocation All,CA_8614,CSWRCB_Water Rights,Irrigation,Fire Protection,S028316,Statement of Div and Use,DANIEL CROWE,...,,,,,,,,,08/06/2020,


In [26]:
print("Writing outputs...")

#write out
outdf100.to_csv(out_alloc, index=False, encoding = "utf-8")

print("Done Water Allocation")

Writing outputs...
Done Water Allocation
